# Setup

In [1]:
import numpy as np
import pandas as pd

# Exercise 1

With the earthquakes.csv file, select all the earthquakes in Japan with a
magnitude of 4.9 or greater using the mb magnitude type.

In [2]:
quakes = pd.read_csv('exercises/earthquakes.csv')

filteredQuakes = quakes\
    .loc[quakes.place.str.contains('Japan')]\
    .loc[quakes.mag >= 4.9]\
    .loc[quakes.magType == 'mb']

filteredQuakes.head()

,mag,magType,time,place,tsunami,parsed_place
1563,4.9,mb,1538977532250,"293km ESE of Iwo Jima, Japan",0,Japan
2576,5.4,mb,1538697528010,"37km E of Tomakomai, Japan",0,Japan
3072,4.9,mb,1538579732490,"15km ENE of Hasaki, Japan",0,Japan
3632,4.9,mb,1538450871260,"53km ESE of Hitachi, Japan",0,Japan


# Exercise 2

Create bins for each full number of earthquake magnitude (for instance, the first bin
is (0, 1], the second is (1, 2], and so on) with the ml magnitude type and count how
many are in each bin.


In [3]:
binnedQuakes = quakes\
        .query("magType == 'ml'")\
        .assign(
                mag_binned=lambda x: pd.cut(
                        x.mag, 
                        bins=np.arange(0, 7)
                )
        )\

binnedQuakes['mag_binned'].value_counts()



(1, 2]    3105
(0, 1]    2207
(2, 3]     862
(3, 4]     122
(4, 5]       2
(5, 6]       1
Name: mag_binned, dtype: int64

# Exercise 3

Using the faang.csv file, group by the ticker and resample to monthly frequency.
Make the following aggregations:
- Mean of the opening price
- Maximum of the high price
- Minimum of the low price
- Mean of the closing price
- Sum of the volume traded

In [4]:
faang = pd.read_csv(
    'exercises/faang.csv', 
    index_col='date',
    parse_dates=True
)

groupedFaang = faang\
    .groupby(['ticker', pd.Grouper(freq='M')])\
    .agg(
        {
        'open': np.mean,
        'high': np.max,
        'low': np.min,
        'close': np.mean,
        'volume': np.sum
        }
    )

groupedFaang.head()

open       high        low      close        volume
ticker date                                                                
AAPL   2018-01-31  43.505357  45.025002  41.174999  43.501309  2.638718e+09
       2018-02-28  41.819079  45.154999  37.560001  41.909737  3.711577e+09
       2018-03-31  43.761786  45.875000  41.235001  43.624048  2.854911e+09
       2018-04-30  42.441310  44.735001  40.157501  42.458572  2.664617e+09
       2018-05-31  46.239091  47.592499  41.317501  46.384205  2.483905e+09

# Exercise 4

Build a crosstab with the earthquake data between the tsunami column and the
magType column. Rather than showing the frequency count, show the maximum
magnitude that was observed for each combination. Put the magnitude type along
the columns.

In [5]:
crosstabQuakes = pd.crosstab(
    index=quakes.tsunami, 
    columns=quakes.magType,
    colnames=['magType'], 
    values=quakes.mag, 
    aggfunc=np.max
)

crosstabQuakes.head()

magType,mb,mb_lg,md,mh,ml,ms_20,mw,mwb,mwr,mww
tsunami,,,,,,,,,,
0,5.6,3.5,4.11,1.1,4.2,NaN,3.83,5.8,4.8,6.0
1,6.1,NaN,NaN,NaN,5.1,5.7,4.41,NaN,NaN,7.5


# Exercise 5

Calculate the rolling 60-day aggregations of the OHLC data by ticker for the
FAANG data. Use the same aggregations as exercise 3.

In [6]:
rollingFaang = faang\
    .groupby('ticker')\
    .rolling('60D')\
    .agg(
        {
        'open': np.mean,
        'high': np.max,
        'low': np.min,
        'close': np.mean,
        'volume': np.sum
        }
    )

rollingFaang.loc[('FB')].head()


,open,high,low,close,volume
date,,,,,
2018-01-02,177.679993,181.580002,177.550003,181.419998,18151900.0
2018-01-03,179.779999,184.779999,177.550003,183.044998,35038500.0
2018-01-04,181.486664,186.210007,177.550003,183.473333,48919400.0
2018-01-05,182.512497,186.899994,177.550003,184.317501,62493900.0
2018-01-08,183.449997,188.899994,177.550003,185.110001,80488600.0


# Exercise 6

Create a pivot table of the FAANG data that compares the stocks. Put the ticker in
the rows and show the averages of the OHLC and volume traded data.

In [7]:
pivotFaang = faang.pivot_table(
    index='ticker',
    aggfunc=np.mean
)

pivotFaang


,close,high,low,open,volume
ticker,,,,,
AAPL,47.263357,47.748526,46.795877,47.277859,1.360803e+08
AMZN,1641.726176,1662.839839,1619.840519,1644.072709,5.648994e+06
FB,171.510956,173.613347,169.303148,171.472948,2.765860e+07
GOOG,1113.225134,1125.777606,1101.001658,1113.554101,1.741965e+06
NFLX,319.290319,325.219322,313.187330,319.620558,1.146962e+07


# Exercise 7

Calculate the Z-scores for each numeric column of Amazon's data ( ticker is
AMZN) in Q4 2018 using apply().

In [8]:
zscoresAMZN = faang.loc['2018-Q4']\
    .query('ticker==\'AMZN\'')\
    .drop(columns='ticker')\
    .apply(
        lambda x: (x - x.mean())/x.std()
    )

zscoresAMZN.head()

,high,low,open,close,volume
date,,,,,
2018-10-01,2.368006,2.502113,2.337813,2.385848,-1.630411
2018-10-02,2.227302,2.247433,2.190795,2.155037,-0.861879
2018-10-03,2.058955,2.139987,2.068570,2.025489,-0.920345
2018-10-04,1.819474,1.781561,1.850048,1.722816,-0.126582
2018-10-05,1.628173,1.554416,1.642819,1.584748,-0.298771


# Exercise 8

Add event descriptions:

* Create a dataframe with the following three columns: ticker , date , and event. The columns should have the following values:

    - ticker : 'FB'
    - date : ['2018-07-25', '2018-03-19', '2018-03-20']
    - event : ['Disappointing user growth announced after close.', 'Cambridge Analytica story', 'FTC investigation']

* Set the index to ['date', 'ticker'] .

* Merge this data with the FAANG data using an outer join.

In [9]:
fbEvents = pd.DataFrame(
    {
        'ticker' : 'FB',
        'date' : [
            '2018-07-25', 
            '2018-03-19', 
            '2018-03-20'
            ],
        'event' : [
            'Disappointing user growth announced after close.', 
            'Cambridge Analytica story', 
            'FTC investigation'
            ]
    }
)\
.assign(
    date=lambda x: pd.to_datetime(x.date)
)\
.set_index(['date', 'ticker'])

reindexedFaang = faang\
    .reset_index()\
    .set_index(['date', 'ticker'])

mergedFb = reindexedFaang.join(
    fbEvents,
    how='outer'
    )

# mergedFb.loc[('2018-07-25', )]
mergedFb.loc[('2018-07-25', 'FB')]



high                                            218.619995
low                                             214.270004
open                                            215.720001
close                                           217.500000
volume                                     58954200.000000
event     Disappointing user growth announced after close.
Name: (2018-07-25 00:00:00, FB), dtype: object

# Exercise 9

Use the transform() method on the FAANG data to represent all the
values in terms of the first date in the data. To do so, divide all the values for
each ticker by the values for the first date in the data for that ticker. This is
referred to as an index, and the data for the first date is the base ( https://
ec.europa.eu/eurostat/statistics-explained/index.php/
Beginners:Statistical_concept_-_Index_and_base_year ). When
data is in this format, we can easily see growth over time. Hint: transform() can
take a function name.

In [10]:
'''
faang2 = faang\
    .groupby('ticker')\
    .assign(
        first_index_value=lambda x: x.transform(), 
        percentage_of_first=lambda x: x.div(x.first_index_value)
    )\
'''
faangReindexed = faang.reset_index().set_index(['ticker', 'date'])

numerator = faangReindexed
denominator =     faangReindexed\
    .groupby(level='ticker')\
    .transform('first')
faangFirstIndex = numerator / denominator

# View 3 rows of the result per ticker
faangFirstIndex\
    .groupby(level='ticker')\
    .agg('head', 3)



high       low      open     close    volume
ticker date                                                        
FB     2018-01-02  1.000000  1.000000  1.000000  1.000000  1.000000
       2018-01-03  1.017623  1.021290  1.023638  1.017914  0.930294
       2018-01-04  1.025498  1.036891  1.040635  1.016040  0.764708
AAPL   2018-01-02  1.000000  1.000000  1.000000  1.000000  1.000000
       2018-01-03  1.013059  1.015952  1.013928  0.999826  1.155033
       2018-01-04  1.006790  1.016661  1.013987  1.004470  0.877864
AMZN   2018-01-02  1.000000  1.000000  1.000000  1.000000  1.000000
       2018-01-03  1.013017  1.015199  1.013908  1.012775  1.153758
       2018-01-04  1.021739  1.029175  1.028157  1.017308  1.121581
NFLX   2018-01-02  1.000000  1.000000  1.000000  1.000000  1.000000
       2018-01-03  1.022614  1.031112  1.030342  1.019794  0.783394
       2018-01-04  1.026779  1.043905  1.051504  1.022679  0.549800
GOOG   2018-01-02  1.000000  1.000000  1.000000  1.000000  1.000000
       2018-01-03  1.018136  1.017202  1.015234  1.016413  1.155624
       2018-01-04  1.024959  1.037094  1.037831  1.020094  0.811732

# Exercise 10

The European Centre for Disease Prevention and Control (ECDC) provides
an open dataset on COVID-19 cases called daily number of new reported cases
of COVID-19 by country worldwide ( https://www.ecdc.europa.eu/
en/publications-data/download-todays-data-geographic-
distribution-covid-19-cases-worldwide ). This dataset is updated daily,
but we will use a snapshot that contains data through September 18, 2020. Complete
the following tasks to practice the skills you've learned up to this point in the book:

* Prepare the data:
    - Read in the data in the covid19_cases.csv file.
    - Create a date column by parsing the dateRep column into a datetime.
    - Set the date column as the index.
    - Use the replace() method to update all occurrences of United_States_of_America and United_Kingdom to USA and UK, respectively.
    - Sort the index.

In [24]:
globalCases = pd\
    .read_csv('exercises/covid19_cases.csv')\
    .assign(
        date=lambda x: pd.to_datetime(
            x.dateRep,
            format='%d/%m/%Y'
        )
    )\
    .set_index('date')\
    .replace('United_States_of_America', 'USA')\
    .replace('United_Kingdom', 'UK')\
    .sort_index()

globalCases.head()


,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
date,,,,,,,,,,,,
2019-12-31,31/12/2019,31,12,2019,0,0,Belgium,BE,BEL,11455519.0,Europe,NaN
2019-12-31,31/12/2019,31,12,2019,0,0,Mexico,MX,MEX,127575529.0,America,NaN
2019-12-31,31/12/2019,31,12,2019,0,0,Ecuador,EC,ECU,17373657.0,America,NaN
2019-12-31,31/12/2019,31,12,2019,0,0,Russia,RU,RUS,145872260.0,Europe,NaN
2019-12-31,31/12/2019,31,12,2019,0,0,Netherlands,NL,NLD,17282163.0,Europe,NaN


a) For the five countries with the most cases (cumulative), find the day with the largest number of cases.

In [52]:
'''
top_five_countries = globalCases\
    .groupby('countriesAndTerritories')\
    .cases.sum()\
    .nlargest(5).index
'''
top5Countries = globalCases\
    .groupby('countriesAndTerritories')\
    .cases.sum()\
    .nlargest(5)\
    .index

datesWithMostCases = globalCases\
    .loc[globalCases.countriesAndTerritories.isin(top5Countries)]\
    .groupby('countriesAndTerritories')\
    .cases.idxmax()

datesWithMostCases

countriesAndTerritories
Brazil   2020-07-30
India    2020-09-17
Peru     2020-08-17
Russia   2020-07-18
USA      2020-07-25
Name: cases, dtype: datetime64[ns]

b) Find the 7-day average change in COVID-19 cases for the last week in the data for the five countries with the most cases.

In [74]:
'''pass'''
top5Countries
lastWeekAverage = globalCases\
    .groupby(
        ['countriesAndTerritories', pd.Grouper(freq='1D')]
    )\
    .cases.sum()\
    .unstack(0)\
    .diff()\
    .rolling(7).mean()\
    .last('1W')

lastWeekAverage[top5Countries]


countriesAndTerritories,USA,India,Brazil,Russia,Peru
date,,,,,
2020-09-14,473.714286,181.285714,35.285714,36.285714,73.142857
2020-09-15,1513.000000,1142.857143,697.428571,46.285714,377.571429
2020-09-16,3478.714286,59.571429,3196.285714,61.428571,-65.000000
2020-09-17,-1047.000000,308.428571,143.428571,810.000000,-29.428571
2020-09-18,865.714286,-18.142857,-607.714286,-688.428571,-227.571429
2020-09-19,306.857143,-604.714286,-560.142857,57.285714,-41.285714


c) Find the first date that each country other than China had cases.

In [128]:
firstCasesByCountry = globalCases\
    .reset_index()\
    .pivot(
        index='date', 
        columns='countriesAndTerritories', 
        values='cases'
    )\
    .drop(columns='China')\
    .fillna(0)\
    .apply(lambda x: x[(x > 0)].idxmin())\
    .sort_values()\

firstCasesByCountry


countriesAndTerritories
Thailand         2020-01-13
Japan            2020-01-15
South_Korea      2020-01-20
USA              2020-01-21
Taiwan           2020-01-21
                    ...    
Lesotho          2020-05-15
Uruguay          2020-05-17
Western_Sahara   2020-06-20
Mali             2020-07-07
Puerto_Rico      2020-09-10
Length: 209, dtype: datetime64[ns]

d) Rank the countries by cumulative cases using percentiles.

In [150]:
'''pass'''
countriesByPercentiles = globalCases\
    .reset_index()\
    .pivot_table(
        columns='countriesAndTerritories', 
        values='cases',
        aggfunc='sum'
    )\
    .T\
    .sort_values(by='cases', ascending=False)\
    .transform('rank', method='max', pct=True)

countriesByPercentiles

,cases
countriesAndTerritories,
USA,1.000000
India,0.995238
Brazil,0.990476
Russia,0.985714
Peru,0.980952
...,...
Greenland,0.023810
Montserrat,0.019048
Falkland_Islands_(Malvinas),0.019048


<hr>
<div style="overflow: hidden; margin-bottom: 10px;">
    <div style="float: left;">
        <a href="./python_101.ipynb">
            <button>Python 101</button>
        </a>
    </div>
    <div style="float: right;">
        <a href="../../solutions/ch_01/solutions.ipynb">
            <button>Solutions</button>
        </a>
        <a href="../ch_02/1-pandas_data_structures.ipynb">
            <button>Chapter 2 &#8594;</button>
        </a>
    </div>
</div>
<hr>